In [1]:

import sys; sys.path.append("../.."); sys.path.append("../../src/")
from visualisation.vtk_tools import *
from visualisation.vtk_plot import *
import vtk

import numpy as np
import pandas as pd
import string

from dash import html, dcc, Input, Output, State, ctx
import plotly.graph_objs as go
from plotly.colors import qualitative
import dash_vtk
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.lines import Line2D
from matplotlib.ticker import ScalarFormatter
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from vtk.util.numpy_support import vtk_to_numpy
from dash_vtk.utils import to_mesh_state, to_volume_state

In [11]:
path = "data/vtpvti/b5/vtpvti/"
file_main =  path + "soil_rx"+str(int(20*10))+ ".vti"
image_data  = read_rect_vtu(file_main)
cell_data = image_data.GetCellData()
pressure_head_array = cell_data.GetArray("pressure head")

print("Number of values:", pressure_head_array.GetNumberOfTuples())
#ugrid  = read_vtu(file_main)

Number of values: 1440


In [12]:
image_data.GetCellData()

<CellData(0x14482c50) at 0x7f62e8d243a0>

In [16]:
num_arrays = cell_data.GetNumberOfArrays()
for i in range(num_arrays):
    array = cell_data.GetArray(i)
    name = array.GetName()
    print(f"- [{i}] {name}")

- [0] water content
- [1] pressure head
- [2] theta (cm3/cm3)


In [7]:

# Use the correct reader for .vti files
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName(file_main)
reader.Update()

image_data = reader.GetOutput()

# Access the cell data array named "pressure head"
cell_data = image_data.GetCellData()
pressure_head_array = cell_data.GetArray("pressure head")

<vtkmodules.vtkCommonCore.vtkTypeFloat64Array(0x13327a80) at 0x7f62e8d24580>

In [ ]:

# Extract scalar range from point data  
print("Number of arrays in cell data vti:",  image_data.GetCellData().GetNumberOfArrays())  
print("Number of arrays in point data vti:",  image_data.GetPointData().GetNumberOfArrays())  
if image_data.GetPointData().GetNumberOfArrays() == 0:
    raise ValueError("No point data arrays found in the VTU file.")

pname = image_data.GetPointData().GetArrayName(0)
array = image_data.GetPointData().GetArray(pname)
q_exud = vtk_to_numpy(array) if array else np.array([])
q_range = [float(q_exud.min()), float(q_exud.max())] if q_exud.size > 0 else [0.0, 1e-6]

print('q_exud', pname, q_range)

# Create volume state from image data
vol_state = to_volume_state(image_data, field_to_encode=pname)
q_range = vol_state["field"]["dataRange"]  # Updated after conversion